In [ ]:
!pip install transformers torch

In [ ]:
sentence = "Hello AI"





input_ids = tokenizer.encode(sentence, add_special_tokens=True)

print(f"Sentence: '{sentence}'")
print(f"Input IDs: {input_ids}")


decoded = tokenizer.convert_ids_to_tokens(input_ids)
print(f"Decoded:   {decoded}")

Sentence: 'Hello AI'
Input IDs: [101, 7592, 9932, 102]
Decoded:   ['[CLS]', 'hello', 'ai', '[SEP]']


In [ ]:
from transformers import BertTokenizer


tokenizer=BertTokenizer.from_pretrained('bert-base-uncased')


text="i love coding"


inputs=tokenizer(text,padding='max_length',max_length=8,truncation=True,return_tensors='pt')


print(inputs['input_ids'])



print(inputs['attention_mask'])



tensor([[  101,  1045,  2293, 16861,   102,     0,     0,     0]])
tensor([[1, 1, 1, 1, 1, 0, 0, 0]])


In [ ]:
import torch.nn as nn

import torch
from transformers import BertModel

class BERTArch(nn.Module):
  def __init__(self,output_dim):
    super(BERTArch,self). __init__()
    self.bert=BertModel.from_pretrained('bert-base-uncased')


    for param in self.bert.parameters():
        param.requires_grad = False

    self.fc=nn.Linear(768,output_dim)



  def forward(self,sent_id,mask):


    output=self.bert(sent_id,attention_mask=mask)
    cls_vector = output.pooler_output


    x = self.fc(cls_vector)
    return x


In [ ]:
import json
import random
import numpy as np
from transformers import BertModel,BertTokenizer


tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

import torch
import torch.nn as nn
from torch.utils.data import DataLoader,Dataset
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')


with open("intents.json",'r' ) as f:
  intents=json.load(f)






tags = []
for intent in intents['intents']:
    tags.append(intent['tag'])
tags = sorted(set(tags))




input_ids = []
attention_masks = []
labels = []


for intent in intents['intents']:
    for pattern in intent['patterns']:


        encoded = tokenizer(
            pattern,
            add_special_tokens=True,
            max_length=20,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )


        input_ids.append(encoded['input_ids'][0])
        attention_masks.append(encoded['attention_mask'][0])


        label_id = tags.index(intent['tag'])
        labels.append(label_id)


input_ids = torch.stack(input_ids)
attention_masks = torch.stack(attention_masks)
labels = torch.tensor(labels)


# print(f"input_ids : {input_ids}")
# print(f"attention_masks : {attention_masks}")
# print(f"labels : {labels}")




class ChatDataset(Dataset):
    def __init__(self, encodings, masks, labels):
        self.encodings = encodings
        self.masks = masks
        self.labels = labels

    def __getitem__(self, idx):
        return {
            'input_ids': self.encodings[idx],
            'attention_mask': self.masks[idx],
            'labels': self.labels[idx]
        }

    def __len__(self):
        return len(self.labels)


dataset = ChatDataset(input_ids, attention_masks, labels)
train_loader = DataLoader(dataset, batch_size=8, shuffle=True)








from torch.optim import AdamW

class BERT_Arch(nn.Module):
    def __init__(self, output_dim):
        super(BERT_Arch, self).__init__()


        self.bert = BertModel.from_pretrained('bert-base-uncased')


        for param in self.bert.parameters():
            param.requires_grad = False


        self.dropout = nn.Dropout(0.1)
        self.relu = nn.ReLU()

        self.fc = nn.Linear(768, output_dim)

    def forward(self, sent_id, mask):

        output = self.bert(sent_id, attention_mask=mask)


        cls_vector = output.pooler_output


        x = self.fc(self.dropout(cls_vector))
        return x


output_dim = len(tags)
model = BERT_Arch(output_dim)
model = model.to(device)


optimizer = AdamW(model.parameters(), lr=1e-3)


cross_entropy = nn.CrossEntropyLoss()





epochs = 300



for epoch in range(epochs):

    total_loss = 0

    for batch in train_loader:


        sent_id = batch['input_ids'].to(device)
        mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)


        model.zero_grad()




        preds = model(sent_id, mask)


        loss = cross_entropy(preds, labels)


        loss.backward()


        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()


        total_loss += loss.item()


    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch+1} / {epochs} | Loss: {avg_loss:.4f}")


output_data = {
    "model_state": model.state_dict(),
    "output_dim": output_dim,
    "tags": tags
}

torch.save(output_data, "bert_data.pth")
print("Training Complete. Brain saved to 'bert_data.pth'")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Training started for 300 epochs...
Epoch 1 / 300 | Loss: 3.1023
Epoch 2 / 300 | Loss: 2.8361
Epoch 3 / 300 | Loss: 2.7074
Epoch 4 / 300 | Loss: 2.6607
Epoch 5 / 300 | Loss: 2.5954
Epoch 6 / 300 | Loss: 2.5181
Epoch 7 / 300 | Loss: 2.4903
Epoch 8 / 300 | Loss: 2.5194
Epoch 9 / 300 | Loss: 2.4856
Epoch 10 / 300 | Loss: 2.4197
Epoch 11 / 300 | Loss: 2.3493
Epoch 12 / 300 | Loss: 2.3974
Epoch 13 / 300 | Loss: 2.3575
Epoch 14 / 300 | Loss: 2.3284
Epoch 15 / 300 | Loss: 2.3099
Epoch 16 / 300 | Loss: 2.2839
Epoch 17 / 300 | Loss: 2.2777
Epoch 18 / 300 | Loss: 2.2751
Epoch 19 / 300 | Loss: 2.2292
Epoch 20 / 300 | Loss: 2.2516
Epoch 21 / 300 | Loss: 2.2282
Epoch 22 / 300 | Loss: 2.2341
Epoch 23 / 300 | Loss: 2.1619
Epoch 24 / 300 | Loss: 2.1444
Epoch 25 / 300 | Loss: 2.1268
Epoch 26 / 300 | Loss: 2.1270
Epoch 27 / 300 | Loss: 2.1248
Epoch 28 / 300 | Loss: 2.0184
Epoch 29 / 300 | Loss: 2.0527
Epoch 30 / 300 | Loss: 2.0250
Epoch 31 / 300 | Loss: 2.0333
Epoch 32 / 300 | Loss: 1.9879
Epoch 33 / 300